In [1]:
import numpy as np
import h5py

import e2c as e2c_util

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1", "2", "3"

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras import losses


# GPU memory management
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.75

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
def reconstruction_loss(x, t_decoded):
    '''
    Reconstruction loss for the plain VAE
    '''
    v = 0.1
    # return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v) + 0.5*K.log(2*np.pi*v), axis=-1))
    return K.mean(K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2 / (2*v), axis=-1))
    # return K.sum((K.batch_flatten(x) - K.batch_flatten(t_decoded)) ** 2, axis=-1)


def l2_reg_loss(qm):
    # 0.5 * (torch.log(pv) - torch.log(qv) + qv / pv + (qm - pm).pow(2) / pv - 1)
    # -0.5 * K.sum(1 + t_log_var - K.square(t_mean) - K.exp(t_log_var), axis=-1)
#     kl = -0.5 * (1 - p_logv + q_logv - K.exp(q_logv) / K.exp(p_logv) - K.square(qm - pm) / K.exp(p_logv))
    l2_reg = 0.5*K.square(qm)
    return K.mean(K.sum(l2_reg, axis=-1))


def get_flux_loss(m, state, state_pred):
    '''
    @params:  state, state_pred shape (batch_size, 60, 60, 2)
              p, p_pred shape (batch_size, 60, 60, 1)
              m shape (batch_size, 60, 60, 1)
    
    @return:  loss_flux: scalar
    
    Only consider discrepancies in total flux, not in phases (saturation not used) 
    '''
    
    perm = K.exp(m)
    p = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)

    #print(K.int_shape(xxx))
    
    tran_x = 1./perm[:, 1:, ...] + 1./perm[:, :-1, ...]
    tran_y = 1./perm[:, :, 1:, ...] + 1./perm[:, :, :-1, ...]
    flux_x = (p[:, 1:, ...] - p[:, :-1, ...]) / tran_x
    flux_y = (p[:, :, 1:, :] - p[:, :, :-1, :]) / tran_y
    flux_x_pred = (p_pred[:, 1:, ...] - p_pred[:, :-1, ...]) / tran_x
    flux_y_pred = (p_pred[:, :, 1:, :] - p_pred[:, :, :-1, :]) / tran_y

    loss_x = K.sum(K.abs(K.batch_flatten(flux_x) - K.batch_flatten(flux_x_pred)), axis=-1)
    loss_y = K.sum(K.abs(K.batch_flatten(flux_y) - K.batch_flatten(flux_y_pred)), axis=-1)

    loss_flux = K.mean(loss_x + loss_y)
    return loss_flux

In [4]:
def get_binary_sat_loss(state, state_pred):
    
    sat_threshold = 0.105
    sat = K.expand_dims(state[:, :, :, 0], -1)
    sat_pred = K.expand_dims(state_pred[:, :, :, 0], -1)
    
    
    sat_bool = K.greater_equal(sat, sat_threshold) #will return boolean values
    sat_bin = K.cast(sat_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
    sat_pred_bool = K.greater_equal(sat_pred, sat_threshold) #will return boolean values
    sat_pred_bin = K.cast(sat_pred_bool, dtype=K.floatx()) #will convert bool to 0 and 1  
    
#     binary_loss = K.sum(K.abs(K.batch_flatten(sat_bin) - K.batch_flatten(sat_pred_bin)), axis=-1)
    
    binary_loss = losses.binary_crossentropy(sat_bin, sat_pred_bin)
    return K.mean(binary_loss)

In [5]:
def get_well_bhp_loss(state, state_pred, prod_well_loc):
    '''
    @params: state: shape (batch_size, 60, 60, 2)
             state_pred: shape (batch_size, 60, 60, 2)
             prod_well_loc: shape (5, 2)
             
    p_true: shape (batch_size, 60, 60, 1)
    p_pred: shape (batch_size, 60, 60, 1)
    
    @return: bhp_loss: scalar
    '''
    
    p_true = K.expand_dims(state[:, :, :, 1], -1)
    p_pred = K.expand_dims(state_pred[:, :, :, 1], -1)
    
    print(K.int_shape(prod_well_loc))
#     print(K.int_shape(prod_well_loc[0, :, 0]))
#     print(K.int_shape(prod_well_loc[0, :, 1]))
    print(K.int_shape(prod_well_loc[:, 0]))
    print(K.int_shape(prod_well_loc[:, 1]))
    print(prod_well_loc.shape[0])
    bhp_loss = 0
    for i in range(prod_well_loc.shape[0]):
        bhp_loss += K.mean(K.abs(p_true[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :] - p_pred[:, prod_well_loc[i, 1], prod_well_loc[i, 0], :]))
#     bhp_loss = K.sum(p_true[:, prod_well_loc[:, 0], prod_well_loc[:, 1], :] - p_pred[:, prod_well_loc[:,0], prod_well_loc[:,1], :])
    
    return bhp_loss

In [6]:
def create_e2c(latent_dim, u_dim, input_shape, sigma=0.0):
    '''
    Creates a E2C.

    Args:
        latent_dim: dimensionality of latent space
        return_kl_loss_op: whether to return the operation for
                           computing the KL divergence loss.

    Returns:
        The VAE model. If return_kl_loss_op is True, then the
        operation for computing the KL divergence loss is
        additionally returned.
    '''

    encoder_ = e2c_util.create_encoder(latent_dim, input_shape, sigma=sigma)
    decoder_ = e2c_util.create_decoder(latent_dim, input_shape)
    transition_ = e2c_util.create_trans(latent_dim, u_dim)

    return encoder_, decoder_, transition_

In [7]:
# Create plain E2C model and associated loss operations

################### case specification ######################

# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
data_dir = '/data3/Astro/personal/zjin/datasets/9W_MS_BHP_RATE_GAU/'
# data_dir = '/data3/Astro/personal/zjin/datasets/7W_CHA/'

output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/'

# case_name = '9w_bhp'
# case_name = '9w_bhp_rate'
case_name = '9w_ms_bhp_rate'
# case_name = '7w_cha'

# case_suffix = '_single_out_rel_2'
case_suffix = '_fix_wl_rel_8'

train_suffix = '_with_p'

# model_suffix = '_flux_loss'
# model_suffix = '_ae_no_l2_ep_10'
model_suffix = '_no_fl'


n_train_run = 300
n_eval_run = 100
num_t = 20 
# dt = 200 // num_t
dt = 100
n_train_step = n_train_run * num_t
n_eval_step = n_eval_run * num_t


train_file = case_name + '_e2c_train' + case_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat' %(n_train_step, dt, num_t, n_train_run)
eval_file = case_name + '_e2c_eval' + case_suffix + train_suffix +'_n%d_dt%dday_nt%d_nrun%d.mat' %(n_eval_step, dt, num_t, n_eval_run)

#################### model specification ##################
epoch = 10
batch_size = 4
learning_rate = 1e-4
latent_dim = 50

u_dim = 9*2 # control dimension

# load data
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
hf_r.close()

num_train = state_t_train.shape[0]
# dt_train = np.ones((num_train,1)) # dt=20days, normalized to 1

hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
hf_r.close()

num_eval = state_t_eval.shape[0]
# dt_eval = np.ones((num_eval, 1)) # dt=20days, normalized to 1

In [8]:
# m = np.loadtxt("/data/cees/zjin/lstm_rom/sim_runs/case6_9w_bhp_rate_ms_h5/template/logk1.dat")
m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/logk1.dat") # Gaussian
# m = np.loadtxt("/data3/Astro/personal/zjin/sim_runs/case9_cha/template/logk1.dat") # channelized

m = m.reshape(60, 60, 1)
print('m shape is ', m.shape)
#     m_tf = K.placeholder((batch_size, 60, 60 ,1))
m_tf = Input(shape=(60, 60, 1))


m_eval = np.repeat(np.expand_dims(m, axis = 0), state_t_eval.shape[0], axis = 0)
print("m_eval shape is ", m_eval.shape)
m = np.repeat(np.expand_dims(m,axis = 0), state_t_train.shape[0], axis = 0)
print("m shape is ", m.shape)

m shape is  (60, 60, 1)
m_eval shape is  (2000, 60, 60, 1)
m shape is  (6000, 60, 60, 1)


In [9]:
well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case8_9w_bhp_rate_ms_gau/template/well_loc00.dat'
# well_loc_file = '/data3/Astro/personal/zjin/sim_runs/case9_cha/template/well_loc00.dat'
well_loc = np.loadtxt(well_loc_file).astype(int)
num_prod = well_loc[0,0]
num_inj = well_loc[0,1]
num_well = num_prod+num_inj
print(num_inj, num_prod)

prod_loc = well_loc[1:num_prod+1,:]
print("prod_loc:\n{}".format(prod_loc))
# inj_loc = well_loc[num_prod+1:,:]
# print("inj_loc:\n{}".format(inj_loc))

print(prod_loc.shape)

print('prod_loc shape is ', prod_loc.shape)
# prod_loc_tf = Input(shape=(5, 2), dtype ='int32')
prod_loc_tf = tf.placeholder(tf.int32, shape=(num_prod,2))

# prod_loc_eval = np.repeat(np.expand_dims(prod_loc, axis = 0), state_t_eval.shape[0], axis = 0)
# print("prod_loc_eval shape is ", prod_loc_eval.shape)
# prod_loc = np.repeat(np.expand_dims(prod_loc,axis = 0), state_t_train.shape[0], axis = 0)
# print("prod_loc shape is ", prod_loc.shape)

4 5
prod_loc:
[[10 10]
 [ 8 52]
 [30 30]
 [50 10]
 [50 50]]
(5, 2)
prod_loc shape is  (5, 2)


In [10]:
# import importlib
# importlib.reload(e2c_util)


In [10]:
# Construct E2C
input_shape = (60, 60, 2)

#############################################
# here we use a UAE framework, sigma = 0.0001
#############################################
encoder, decoder, transition = create_e2c(latent_dim, u_dim, input_shape, sigma=0.0) 


xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=(u_dim, ))
dt = Input(shape=(1,))

zt = encoder(xt)
xt_rec = decoder(zt)

zt1 = encoder(xt1)

zt1_pred = transition([zt, ut, dt])
xt1_pred = decoder(zt1_pred)

# Compute loss
loss_rec_t = reconstruction_loss(xt, xt_rec)
loss_rec_t1 = reconstruction_loss(xt1, xt1_pred)

loss_flux_t = get_flux_loss(m_tf, xt, xt_rec) / 1000.
loss_flux_t1 = get_flux_loss(m_tf, xt1, xt1_pred) / 1000.

binary_sat_loss_t = get_binary_sat_loss(xt, xt_rec) * 1
binary_sat_loss_t1 = get_binary_sat_loss(xt1, xt1_pred) * 1

loss_prod_bhp_t = get_well_bhp_loss(xt, xt_rec, prod_loc_tf) * 20
loss_prod_bhp_t1 = get_well_bhp_loss(xt1, xt1_pred, prod_loc_tf) * 20

loss_l2_reg = l2_reg_loss(zt)  # log(1.) = 0.
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_kl + binary_sat_loss_t + binary_sat_loss_t1
# loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # JCP 2019 Gaussian case
# loss_bound = loss_rec_t + loss_rec_t1  + loss_flux_t + loss_flux_t1 + loss_prod_bhp_t + loss_prod_bhp_t1 # UAE
loss_bound = loss_rec_t + loss_rec_t1 + loss_l2_reg # no flux/bhp loss comparison



# Use zt_logvar to approximate zt1_logvar_pred
loss_trans = l2_reg_loss(zt1_pred - zt1)
# loss_trans = kl_normal_loss(zt1_mean_pred, zt1_logvar_pred, zt1_mean, zt1_logvar)


trans_loss_weight = 1.0 # lambda in E2C paper Eq. (11)
loss = loss_bound + trans_loss_weight * loss_trans

(5, 2)
(5,)
(5,)
5
(5, 2)
(5,)
(5,)
5


In [11]:
# Optimization
opt = Adam(lr=learning_rate)

trainable_weights = encoder.trainable_weights + decoder.trainable_weights + transition.trainable_weights

updates = opt.get_updates(loss, trainable_weights)

# iterate = K.function([xt, ut, xt1, m_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, binary_sat_loss_t, binary_sat_loss_t1], updates=updates)
iterate = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss, loss_rec_t, loss_rec_t1, loss_l2_reg, loss_trans, loss_flux_t, loss_flux_t1, loss_prod_bhp_t, loss_prod_bhp_t1], updates=updates)

eval_loss = K.function([xt, ut, xt1, m_tf, prod_loc_tf, dt], [loss])

num_batch = int(num_train/batch_size)

for e in range(epoch):
    for ib in range(num_batch):
        ind0 = ib * batch_size
        state_t_batch  = state_t_train[ind0:ind0+batch_size, ...]
        state_t1_batch = state_t1_train[ind0:ind0 + batch_size, ...]
        bhp_batch      = bhp_train[ind0:ind0 + batch_size, ...]
        m_batch        = m[ind0:ind0 + batch_size, ...]
        dt_batch       = dt_train[ind0:ind0 + batch_size, ...]
        
#         print(state_t_batch.shape)
#         print(state_t1_batch.shape)
#         print(bhp_batch.shape)
#         print(m_batch.shape)
#         print(dt_batch.shape)
        

        output = iterate([state_t_batch, bhp_batch, state_t1_batch, m_batch, prod_loc, dt_batch])

        # tf.session.run(feed_dict={xt: sat_t_batch, ut: bhp_batch, xt1: sat_t1_batch}, ...
        #                fetches= [loss, loss_rec_t, loss_rec_t1, loss_kl, loss_trans, updates])
        # But output tensor for the updates operation is not returned

        if ib % 10 == 0:
            print('Epoch %d/%d, Batch %d/%d, Loss %f, Loss rec %f, loss rec t1 %f, loss kl %f, loss_trans %f, loss flux %f, loss flux t1 %f, prod bhp loss %f, prod bhp loss t1 %f'
                  % (e+1, epoch, ib+1, num_batch, output[0], output[1], output[2], output[3], output[4], output[5], output[6], output[7], output[8]))
    eval_loss_val = eval_loss([state_t_eval, bhp_eval, state_t1_eval, m_eval, prod_loc, dt_eval])
    
    print('====================================================')
    print('\n')
    print('Epoch %d/%d, Train loss %f, Eval loss %f' % (e + 1, epoch, output[0], eval_loss_val[0]))
    print('\n')
    print('====================================================')

encoder.save_weights(output_dir + 'e2c_encoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
decoder.save_weights(output_dir + 'e2c_decoder_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                     % (num_train, latent_dim, learning_rate, epoch))
transition.save_weights(output_dir + 'e2c_transition_dt_' + case_name + case_suffix + train_suffix + model_suffix + '_nt%d_l%d_lr%.0e_ep%d.h5' \
                        % (num_train, latent_dim, learning_rate, epoch))

Epoch 1/10, Batch 1/1500, Loss 15984.851562, Loss rec 7886.440430, loss rec t1 8098.192871, loss kl 0.101675, loss_trans 0.117088, loss flux 83.056190, loss flux t1 83.669220, prod bhp loss 36.378448, prod bhp loss t1 36.694664
Epoch 1/10, Batch 11/1500, Loss 5992.577148, Loss rec 2097.992432, loss rec t1 3847.118408, loss kl 19.706306, loss_trans 27.759789, loss flux 266.475891, loss flux t1 194.226944, prod bhp loss 11.651307, prod bhp loss t1 12.726505
Epoch 1/10, Batch 21/1500, Loss 7068.491211, Loss rec 2490.350098, loss rec t1 4539.361328, loss kl 17.630568, loss_trans 21.149618, loss flux 266.251373, loss flux t1 177.400085, prod bhp loss 10.569798, prod bhp loss t1 16.428070
Epoch 1/10, Batch 31/1500, Loss 2916.270508, Loss rec 1094.222534, loss rec t1 1778.478882, loss kl 17.962318, loss_trans 25.606663, loss flux 297.260956, loss flux t1 232.700317, prod bhp loss 11.696917, prod bhp loss t1 9.393800
Epoch 1/10, Batch 41/1500, Loss 2218.995361, Loss rec 918.524048, loss rec t1

Epoch 1/10, Batch 361/1500, Loss 341.883942, Loss rec 184.935364, loss rec t1 142.253784, loss kl 7.426806, loss_trans 7.267964, loss flux 214.079590, loss flux t1 200.928101, prod bhp loss 12.630579, prod bhp loss t1 10.388853
Epoch 1/10, Batch 371/1500, Loss 342.085022, Loss rec 152.772797, loss rec t1 173.594711, loss kl 7.809407, loss_trans 7.908116, loss flux 212.424576, loss flux t1 208.056488, prod bhp loss 9.710069, prod bhp loss t1 8.891196
Epoch 1/10, Batch 381/1500, Loss 272.859406, Loss rec 145.891922, loss rec t1 114.842079, loss kl 6.206408, loss_trans 5.918983, loss flux 196.546860, loss flux t1 183.404144, prod bhp loss 10.713328, prod bhp loss t1 8.667349
Epoch 1/10, Batch 391/1500, Loss 280.694458, Loss rec 154.576752, loss rec t1 115.095856, loss kl 5.631876, loss_trans 5.389989, loss flux 188.499786, loss flux t1 184.014496, prod bhp loss 8.998373, prod bhp loss t1 11.287465
Epoch 1/10, Batch 401/1500, Loss 361.743103, Loss rec 164.465622, loss rec t1 184.364151, lo

Epoch 1/10, Batch 731/1500, Loss 95.410728, Loss rec 48.627960, loss rec t1 36.642651, loss kl 5.727733, loss_trans 4.412384, loss flux 194.607437, loss flux t1 188.878036, prod bhp loss 7.925097, prod bhp loss t1 7.606214
Epoch 1/10, Batch 741/1500, Loss 118.048470, Loss rec 72.609024, loss rec t1 38.708839, loss kl 3.576329, loss_trans 3.154277, loss flux 166.608978, loss flux t1 165.784988, prod bhp loss 8.841125, prod bhp loss t1 8.509793
Epoch 1/10, Batch 751/1500, Loss 124.206291, Loss rec 73.714005, loss rec t1 42.165573, loss kl 4.745289, loss_trans 3.581426, loss flux 187.421661, loss flux t1 175.138168, prod bhp loss 10.796148, prod bhp loss t1 8.963283
Epoch 1/10, Batch 761/1500, Loss 113.913040, Loss rec 61.026108, loss rec t1 44.748650, loss kl 4.495509, loss_trans 3.642778, loss flux 181.206207, loss flux t1 182.793015, prod bhp loss 8.528435, prod bhp loss t1 9.276871
Epoch 1/10, Batch 771/1500, Loss 122.727013, Loss rec 56.634495, loss rec t1 58.183907, loss kl 4.342144

Epoch 1/10, Batch 1101/1500, Loss 62.054195, Loss rec 25.986004, loss rec t1 30.953678, loss kl 3.445704, loss_trans 1.668811, loss flux 153.839798, loss flux t1 155.014038, prod bhp loss 8.981791, prod bhp loss t1 9.288240
Epoch 1/10, Batch 1111/1500, Loss 67.354408, Loss rec 31.492332, loss rec t1 31.952280, loss kl 2.421220, loss_trans 1.488582, loss flux 143.550934, loss flux t1 151.393799, prod bhp loss 8.929376, prod bhp loss t1 8.102770
Epoch 1/10, Batch 1121/1500, Loss 42.320412, Loss rec 19.477999, loss rec t1 18.795515, loss kl 2.816110, loss_trans 1.230789, loss flux 142.505768, loss flux t1 139.748352, prod bhp loss 7.897251, prod bhp loss t1 5.783824
Epoch 1/10, Batch 1131/1500, Loss 71.595177, Loss rec 24.247372, loss rec t1 44.219322, loss kl 2.142616, loss_trans 0.985871, loss flux 135.069000, loss flux t1 135.911179, prod bhp loss 7.016975, prod bhp loss t1 6.055140
Epoch 1/10, Batch 1141/1500, Loss 48.923370, Loss rec 25.316849, loss rec t1 20.367104, loss kl 2.096583

Epoch 1/10, Batch 1471/1500, Loss 27.684883, Loss rec 10.588961, loss rec t1 14.303827, loss kl 2.100278, loss_trans 0.691819, loss flux 127.891975, loss flux t1 129.555557, prod bhp loss 6.908004, prod bhp loss t1 6.553759
Epoch 1/10, Batch 1481/1500, Loss 38.838394, Loss rec 15.307008, loss rec t1 21.405289, loss kl 1.487575, loss_trans 0.638524, loss flux 121.038521, loss flux t1 120.924911, prod bhp loss 6.373395, prod bhp loss t1 5.719404
Epoch 1/10, Batch 1491/1500, Loss 29.905460, Loss rec 11.535770, loss rec t1 15.611245, loss kl 2.075602, loss_trans 0.682845, loss flux 126.600037, loss flux t1 126.565704, prod bhp loss 6.458556, prod bhp loss t1 6.006122


Epoch 1/10, Train loss 24.320267, Eval loss 260.618073


Epoch 2/10, Batch 1/1500, Loss 33.001671, Loss rec 11.771427, loss rec t1 18.600433, loss kl 1.860639, loss_trans 0.769173, loss flux 121.815628, loss flux t1 127.011482, prod bhp loss 8.107897, prod bhp loss t1 7.505421
Epoch 2/10, Batch 11/1500, Loss 25.503262, Loss 

...

Epoch 9/10, Batch 1151/1500, Loss 10.060957, Loss rec 4.277511, loss rec t1 5.404096, loss kl 0.353953, loss_trans 0.025398, loss flux 55.299175, loss flux t1 56.863346, prod bhp loss 1.536803, prod bhp loss t1 1.840878
Epoch 9/10, Batch 1161/1500, Loss 11.982950, Loss rec 4.396759, loss rec t1 7.211087, loss kl 0.334397, loss_trans 0.040708, loss flux 57.174503, loss flux t1 56.501770, prod bhp loss 2.197431, prod bhp loss t1 1.660290
Epoch 9/10, Batch 1171/1500, Loss 8.410143, Loss rec 4.984178, loss rec t1 3.013968, loss kl 0.381989, loss_trans 0.030008, loss flux 54.287434, loss flux t1 54.952095, prod bhp loss 1.917252, prod bhp loss t1 1.594422
Epoch 9/10, Batch 1181/1500, Loss 7.920346, Loss rec 3.059228, loss rec t1 4.337246, loss kl 0.502256, loss_trans 0.021616, loss flux 53.975090, loss flux t1 53.787041, prod bhp loss 1.599293, prod bhp loss t1 1.703554
Epoch 9/10, Batch 1191/1500, Loss 8.577233, Loss rec 4.433146, loss rec t1 3.805780, loss kl 0.307896, loss_trans 0.030412

Epoch 10/10, Batch 21/1500, Loss 7.764380, Loss rec 4.350331, loss rec t1 2.942354, loss kl 0.429154, loss_trans 0.042541, loss flux 50.608673, loss flux t1 53.781410, prod bhp loss 2.374400, prod bhp loss t1 2.252450
Epoch 10/10, Batch 31/1500, Loss 6.736474, Loss rec 2.778615, loss rec t1 3.666278, loss kl 0.259899, loss_trans 0.031682, loss flux 45.882885, loss flux t1 51.261261, prod bhp loss 1.514678, prod bhp loss t1 1.513051
Epoch 10/10, Batch 41/1500, Loss 7.701133, Loss rec 3.343025, loss rec t1 3.968714, loss kl 0.363935, loss_trans 0.025460, loss flux 52.198948, loss flux t1 51.911079, prod bhp loss 2.266937, prod bhp loss t1 2.221534
Epoch 10/10, Batch 51/1500, Loss 5.805561, Loss rec 2.982806, loss rec t1 2.352540, loss kl 0.431796, loss_trans 0.038419, loss flux 56.072472, loss flux t1 55.845181, prod bhp loss 1.844853, prod bhp loss t1 1.996524
Epoch 10/10, Batch 61/1500, Loss 4.264460, Loss rec 1.916525, loss rec t1 2.026171, loss kl 0.285337, loss_trans 0.036428, loss 

Epoch 10/10, Batch 401/1500, Loss 9.345460, Loss rec 4.401237, loss rec t1 4.481275, loss kl 0.418754, loss_trans 0.044195, loss flux 44.578659, loss flux t1 55.732620, prod bhp loss 1.033966, prod bhp loss t1 1.901082
Epoch 10/10, Batch 411/1500, Loss 4.478854, Loss rec 2.428955, loss rec t1 1.681499, loss kl 0.354288, loss_trans 0.014112, loss flux 48.731895, loss flux t1 49.205784, prod bhp loss 1.549957, prod bhp loss t1 1.642901
Epoch 10/10, Batch 421/1500, Loss 3.458042, Loss rec 1.347180, loss rec t1 1.735834, loss kl 0.342526, loss_trans 0.032502, loss flux 49.414227, loss flux t1 49.946014, prod bhp loss 1.588411, prod bhp loss t1 1.501513
Epoch 10/10, Batch 431/1500, Loss 3.209401, Loss rec 1.364517, loss rec t1 1.524589, loss kl 0.280777, loss_trans 0.039517, loss flux 48.879627, loss flux t1 48.519283, prod bhp loss 1.751741, prod bhp loss t1 1.762407
Epoch 10/10, Batch 441/1500, Loss 3.609597, Loss rec 1.403598, loss rec t1 1.832153, loss kl 0.329263, loss_trans 0.044583, 

Epoch 10/10, Batch 781/1500, Loss 4.063189, Loss rec 2.348849, loss rec t1 1.342103, loss kl 0.343272, loss_trans 0.028964, loss flux 49.759304, loss flux t1 48.826885, prod bhp loss 1.384422, prod bhp loss t1 1.509674
Epoch 10/10, Batch 791/1500, Loss 3.331378, Loss rec 1.447203, loss rec t1 1.571674, loss kl 0.277458, loss_trans 0.035043, loss flux 49.103596, loss flux t1 50.731392, prod bhp loss 1.234806, prod bhp loss t1 1.314198
Epoch 10/10, Batch 801/1500, Loss 13.800155, Loss rec 7.270586, loss rec t1 5.996296, loss kl 0.516263, loss_trans 0.017010, loss flux 54.148716, loss flux t1 55.032330, prod bhp loss 2.827372, prod bhp loss t1 2.883366
Epoch 10/10, Batch 811/1500, Loss 17.825871, Loss rec 8.806471, loss rec t1 8.718080, loss kl 0.273502, loss_trans 0.027818, loss flux 50.307343, loss flux t1 51.039242, prod bhp loss 2.978478, prod bhp loss t1 3.065294
Epoch 10/10, Batch 821/1500, Loss 7.089635, Loss rec 3.259995, loss rec t1 3.363499, loss kl 0.451368, loss_trans 0.014773

Epoch 10/10, Batch 1161/1500, Loss 6.401443, Loss rec 3.120166, loss rec t1 2.948111, loss kl 0.299508, loss_trans 0.033658, loss flux 51.503700, loss flux t1 50.354923, prod bhp loss 2.148778, prod bhp loss t1 1.697517
Epoch 10/10, Batch 1171/1500, Loss 6.051405, Loss rec 2.989665, loss rec t1 2.687497, loss kl 0.346266, loss_trans 0.027978, loss flux 50.512665, loss flux t1 51.423477, prod bhp loss 1.363058, prod bhp loss t1 1.096618
Epoch 10/10, Batch 1181/1500, Loss 3.439243, Loss rec 1.400754, loss rec t1 1.556455, loss kl 0.462049, loss_trans 0.019986, loss flux 50.450657, loss flux t1 50.507458, prod bhp loss 1.383369, prod bhp loss t1 1.723665
Epoch 10/10, Batch 1191/1500, Loss 3.902457, Loss rec 1.353025, loss rec t1 2.256423, loss kl 0.266582, loss_trans 0.026426, loss flux 48.463188, loss flux t1 51.751472, prod bhp loss 1.533063, prod bhp loss t1 1.355820
Epoch 10/10, Batch 1201/1500, Loss 4.561946, Loss rec 1.740507, loss rec t1 2.441746, loss kl 0.336741, loss_trans 0.042